<a href="https://colab.research.google.com/github/shantanumohile/curiouskid/blob/main/Rail_Length_Web_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
from itertools import combinations

st.title("Rail Utilisation Optimizer 🚂")
st.write("Upload an Excel file with a column of values. The app will group them into triplets, pairs, and singles (<= 170).")

uploaded_file = st.file_uploader("Upload your Excel file", type=["xlsx"])

target = 170

def best_fit_group(values, target, r):
    best_combo = None
    best_sum = -1
    for combo in combinations(values, r):
        s = sum(combo)
        if s <= target and s > best_sum:
            best_sum = s
            best_combo = combo
    return list(best_combo) if best_combo else None, best_sum

def make_groups(values, target, r):
    groups = []
    remaining = values.copy()

    while len(remaining) >= r:
        combo, total = best_fit_group(remaining, target, r)
        if not combo:
            break
        groups.append((combo, total))
        for v in combo:
            remaining.remove(v)
    return groups, remaining

if uploaded_file:
    df = pd.read_excel(uploaded_file, sheet_name="Sheet1")
    values = df["ColumnName"].dropna().tolist()  # change ColumnName to your actual column name

    triplets, remaining_after_triplets = make_groups(values, target, 3)
    pairs, remaining_after_pairs = make_groups(remaining_after_triplets, target, 2)
    singles, remaining_final = make_groups(remaining_after_pairs, target, 1)

    st.subheader("Triplets")
    for g, s in triplets:
        st.write(f"{g} → Sum: {s}")

    st.subheader("Pairs")
    for g, s in pairs:
        st.write(f"{g} → Sum: {s}")

    st.subheader("Singles")
    for g, s in singles:
        st.write(f"{g} → Sum: {s}")

    if remaining_final:
        st.subheader("Unassigned values")
        st.write(remaining_final)
